# Fitting with scipy - solution to bonus exercise

In [ ]:
# import python modules for plotting, fitting
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# for interactive plot
import ipywidgets

In [ ]:
def lorentzian(xx, scale=1.0, center=1.0, hwhm=3.0):
    if hwhm == 0:
        raise ValueError('hwhm of the lorentzian is equal to zero.')
    return scale * hwhm / ((xx-center)**2 + hwhm**2) / np.pi

In [ ]:
xx = np.linspace(-10, 10, 500)
lorentzian_noisy_exo = lorentzian(xx, 3, 4, 0.5) * (
    1. + 0.1*np.random.normal(0,1,500)) + 0.01*np.random.normal(0,1,500)

initial_params = [5.5, 0.0, 0.55]

fig8 = plt.figure()
gs = fig8.add_gridspec(3, 1)
f8_ax1 = fig8.add_subplot(gs[0:2, :])
f8_ax2 = fig8.add_subplot(gs[2, :])
f8_ax1.plot(xx, lorentzian_noisy_exo, label="reference data for exercise")
lines = f8_ax1.plot(xx, lorentzian(xx, *initial_params), label='model to be fitted')
fit_lines = f8_ax1.plot(xx, np.zeros_like(xx), '--', label='fit')
res_lines = f8_ax2.plot(xx, np.zeros_like(xx), label='residuals')
f8_ax1.set_ylabel('lorentzian(x,{},{},{})'.format(*initial_params))
f8_ax1.set_xlabel('x')
f8_ax1.grid()
f8_ax1.legend()
f8_ax2.set_xlabel('x')
f8_ax2.grid()
f8_ax2.legend()

def interactive_plot_exo(scale, center, hwhm):
    lines[0].set_ydata(lorentzian(xx, scale, center, hwhm))
    plt.ylabel('lorentzian(x,{scale},{center},{hwhm})'.
               format(scale=scale, center=center, hwhm=hwhm))

interactive_plot_exo = ipywidgets.interactive(interactive_plot_exo, 
                                          scale=(1.0, 10.0), 
                                          center=(-5.0, 5.0), 
                                          hwhm=(0.1, 1.0))

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    for i, p in enumerate(initial_params):
        interactive_plot_exo.children[i].value = p

# Define reset button and occurring action when clicking on it
reset_button_exo = ipywidgets.Button(description = "Reset")
reset_button_exo.on_click(reset_values)

params_exo = [0, 0, 0]
pcov_exo = [0, 0, 0]

# Capture fit results output
fit_results = ipywidgets.Output()

chosen_method_optim = ipywidgets.RadioButtons(
    options=['lm', 'trf', 'dogbox'],
    value='lm', # Defaults to 'lm'
    description='Method for optimization',
    style={'description_width': 'initial'},
    disabled=False
)

# Define reset button and occurring action when clicking on it
run_fit_button = ipywidgets.Button(description = "Fit!")

# display the interactive plot
display(interactive_plot_exo, reset_button_exo)
display(ipywidgets.HBox([chosen_method_optim, run_fit_button, fit_results]))

def run_fit(button):
    params_exo, pcov_exo = curve_fit(lorentzian, xx, lorentzian_noisy_exo,
                                      method=chosen_method_optim.value,
                                      p0=initial_params)
    fit_results.clear_output()
    with fit_results:
        params_error = np.sqrt(np.diag(pcov_exo))
        print('Values of refined parameters:')
        print('scale:', params_exo[0],'+/-', params_error[0])
        print('center :', params_exo[1],'+/-', params_error[1])
        print('HWHM', params_exo[2],'+/-', params_error[2])
    fit_lines[0].set_ydata(lorentzian(xx, *params_exo))
    res_lines[0].set_ydata(lorentzian_noisy_exo - fit_lines[0].get_ydata())

run_fit_button.on_click(run_fit)